In [ ]:
import yaml 
import glob 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# path 
path = r'C:\Users\jyc13\Documents\Python Scripts\openCV_eyemovement\yolov5\runs\detect\exp5\labels'

# object classes
object_classes = yaml.safe_load(open('object_classes.yaml', 'r'))

# YOLO style: (num_class center_x center_y w, h \n) 모든 값은 resoluion 기준으로 비율

# read all txt files in the path
files = glob.glob(f'{path}/*.txt')
data = []
for file in files:
    n_frame = int(file.split('_')[-1].replace('.txt', ''))
    with open(file, 'rt') as f:
        lines = f.readlines()
        for line in lines:
            splited = line.split(' ')
            num_class = int(splited[0])
            class_name = str(object_classes[num_class])
            center_x = float(splited[1])
            center_y = float(splited[2])
            w = float(splited[3])
            h = float(splited[4])

            frame_data = [n_frame, num_class, class_name, center_x, center_y, w, h]
            data.append(frame_data)

# format Dataframe
df = pd.DataFrame(data)
df.rename(columns = {0:'n_frame'}, inplace = True)
df.rename(columns = {1:'num_class'}, inplace = True)
df.rename(columns = {2:'class_name'}, inplace = True)
df.rename(columns = {3:'center_x'}, inplace = True)
df.rename(columns = {4:'center_y'}, inplace = True)
df.rename(columns = {5:'width'}, inplace = True)
df.rename(columns = {6:'height'}, inplace = True)

df.sort_values(by=['n_frame', 'num_class'], inplace=True, ignore_index=True)

# diff
df['diff_x'] = df.groupby(by=['n_frame'])['center_x'].diff()
df['diff_y'] = df.groupby(by=['n_frame'])['center_y'].diff()
display(df)

In [ ]:
# x-axis
sns.lineplot(x='n_frame', y='center_x', hue='class_name', data=df)
plt.show()

In [ ]:
# y-axis
sns.lineplot(x='n_frame', y='center_y', hue='class_name', data=df)
plt.show()

In [ ]:
# diff_x
sns.lineplot(x='n_frame', y='diff_x', hue='class_name', data=df)
plt.show()

In [ ]:
# diff_y
sns.lineplot(x='n_frame', y='diff_y', hue='class_name', data=df)
plt.show()

In [ ]:
from burst import * 

mouse = Burst(video_name="60 L 20hz 5ms 5s.mp4")
df_onoff = mouse.get_stim_on_frames()
display(df_onoff)

### Run YOLO for all videos

In [1]:
import os 

video_repo = r'D:\Research\SC\DATA\Eye_Movement'
file_list = os.listdir(video_repo)
video_list = [x for x in file_list if x.endswith('.MP4')]

print(f'current video files: {video_list}')

def get_filelist_updated(): # 이미 YOLO 분석 된 비디오 파일 리스트 출력
    filepath = r'C:\Users\jyc13\Documents\Python Scripts\openCV_eyemovement\analysis'
    yoloed_videos = 'yoloed_videos.txt'
    f = open(os.path.join(filepath, yoloed_videos), 'r+')
    lines = f.readlines()
    yoloed_video_list = []

    for line in lines:
        yoloed_video_list.append(line.replace('\n', ''))
    
    f.close()
    
    return yoloed_video_list

yoloed_video_list = get_filelist_updated()

s = set(yoloed_video_list)
videos_to_update = [x for x in video_list if x not in s] # YOLO 분석 안 된 비디오만 리스트로 받기

print(f'videos to be updated: {videos_to_update}')


current video files: ['60 L 20hz 5ms 5s.MP4', '60 L base.MP4', '60 R 20hz 5ms 5s.MP4']
videos to be updated: []


In [2]:
from runyolo import runyolo 

for video in videos_to_update: 
    video_name = video
    exp_train = 'exp20'
    save_name = video_name

    runyolo(video_name=video_name, exp_train=exp_train, save_name=video_name)

    f = open('yoloed_videos.txt', 'a')
    f.write(video_name+'\n')
    f.close()
